In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'..\..'))
sys.path.insert(1, os.path.abspath(r'..\..\..\molass-legacy'))
from molass import get_version as molass_version
from molass_legacy import get_version as legacy_version
assert molass_version(toml_only=True) >= '0.7.5', "Molass version is lower than 0.7.5"
assert legacy_version(toml_only=True) >= '0.4.3', "Legacy version is lower than 0.4.3"

In [ ]:
from molass.Global.Options import set_molass_options
set_molass_options(flowchange='auto')
from molass_data import SAMPLE1
from molass.DataObjects import SecSaxsData as SSD

In [ ]:
ssd = SSD(SAMPLE1)
trimmed_ssd = ssd.trimmed_copy()
trimmed_ssd.set_baseline_method(('linear', 'uvdiff'))
corrected_ssd = trimmed_ssd.corrected_copy()
corrected_ssd.plot_compact(baseline=True);

In [ ]:
decomposition = corrected_ssd.quick_decomposition(num_components=3, debug=False)
pairedranges = decomposition.get_pairedranges(area_ratio=0.7,debug=True)
plot1 = decomposition.plot_components(title="Decomposition of Sample1 with Rg Curve and Paired Ranges",
                                      pairedranges=pairedranges)

In [ ]:
target_i = None    # only for debug output
if target_i is None:
    row_dict = {}
else:
    assert row_dict != {}, "row_dict must be populated for target_i debugging"
print("row_dict=", row_dict)
def v1_lrf_report_debug_util(decomposition, pairedranges):
    from importlib import reload
    from molass.Logging.Utils import setup_notebook_logging
    from molass.Backward.SerialDataProxy import SerialDataProxy
    from molass.Backward.PreviewParams import make_preview_params
    import molass.Backward.ConcTracker
    reload(molass.Backward.ConcTracker)
    from molass.Backward.ConcTracker import ConcTracker
    from molass_legacy._MOLASS.SerialSettings import set_setting
    from molass_legacy.LRF.ConcMatrix import ConcMatrix
    from molass_legacy.Extrapolation.PeakSetSelector import PeakSetSelector
    logger = setup_notebook_logging()
    ssd = decomposition.ssd
    mapping  = ssd.get_mapping()
    sd = SerialDataProxy(ssd, decomposition.mapped_curve)
    preview_data, preview_options = make_preview_params(mapping, sd, pairedranges)

    conc_factor = ssd.get_concfactor()
    concentration_datatype = 2
    conc_tracker = ConcTracker(decomposition, conc_factor, concentration_datatype,
                               jupyter=True, debug=True)
    ecurve = preview_data.mapper.x_curve
    selector = PeakSetSelector(preview_data.cnv_ranges, ecurve, debug=False)

    conc = None
    conc_depend = 2
    lrf_rank = conc_depend      # determined later so as to be consistent with xr_ranks in decomposition
    set_setting('concentration_datatype', concentration_datatype)    # referenced in ConcMatrix
    
    logger.info("pairedranges=%s", pairedranges)

    row = 0
    for i, pr in enumerate(pairedranges):
        if target_i is not None and i != target_i:
            logger.info(f"Skipping Paired Range {i+1}: {pr}")
            continue
        logger.info(f"Processing Paired Range {i+1}: {pr}")
        for j, (start, end) in enumerate(pr.get_fromto_list()):
            stop = end + 1
            logger.info(f"  Range {j+1}: {start} - {end}")
            row_ = row if target_i is None else row_dict[(target_i, j)]
            peakset_info = selector.select_peakset(row_)
            pno, nth, peakset, known_peak_info = peakset_info
            logger.info("%s peakset=%s", (i,j), peakset)
            paired_ranges_ = [pairedranges[k] for k in peakset]
            logger.info("%s In the peakset, paired_ranges_=%s", (i,j), paired_ranges_)
            x = ecurve.x[start:stop]
            cmatrix = ConcMatrix(x, conc,
                                 conc_depend=conc_depend,
                                 conc_factor=conc_factor,
                                 paired_ranges=paired_ranges_)
            C_ = cmatrix.data
            conc_tracker.add_concentration(start, stop, C_, conc_dependence=lrf_rank)
            if target_i is None:
                row_dict[(i, j)] = row
            row += 1
        
    conc_tracker.plot()

v1_lrf_report_debug_util(decomposition, pairedranges)